# 4. Spatio-temporal diversity of Valais and Lavaux 2023 must 

In [1]:
#!mkdir /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal

In [2]:
workdir = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal'
%cd $workdir

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal


In [3]:
%env TMPDIR=/scratch/lfloerl/tmpdata

env: TMPDIR=/scratch/lfloerl/tmpdata


In [4]:
from qiime2 import Visualization
import qiime2 as q2
from qiime2 import Visualization
from qiime2 import Artifact
from qiime2 import Metadata
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from qiime2.plugins.diversity.visualizers import alpha_group_significance


from qiime2.plugins import diversity as q2d

%matplotlib inline

In [5]:
md = Metadata.load('/home/lfloerl/microterroir/Microbiome/Metadata/ITS_Valais_and_Lavaux23.tsv')
taxonomy = q2.Artifact.load('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/taxonomy.qza')

In [6]:
# just use the same rarefaction depth for all sample types
depth = 5000 
threads = 5

In [7]:
%%bash 

# filter the Feature Table to contain the Valais and Lavaux 2023 must samples 
qiime feature-table filter-samples \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/phylum_filtered_table.qza \
    --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Valais_and_Lavaux23.tsv \
    --o-filtered-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_table.qza

qiime feature-table summarize \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_table.qza \
    --o-visualization /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_table.qzv

Saved FeatureTable[Frequency] to: /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_table.qza
Saved Visualization to: /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_table.qzv


depth of 5000: Retained 630’000 (24.93%) features in 126 (98.44%) samples


In [8]:
Visualization.load('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_table.qzv')

<visualization: Visualization uuid: 342af413-1161-49f7-97ea-4b12272e8e13>

# Diversity 

In [10]:
table = q2.Artifact.load('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_table.qza')

In [19]:
core_div_dict = {}
core_div = q2d.actions.core_metrics(table=table,
                                    sampling_depth=depth,
                                    metadata=md, 
                                    n_jobs=threads)
# save outputs
output_dir='Valais_cm{0}/'.format(depth)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for k, v in core_div.__dict__.items():
    if k != '_fields':
        v.save(output_dir + k)
        core_div_dict[k] = v

/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  series[missing.index] = missing
/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan na

In [20]:
Visualization.load(f'{workdir}/{output_dir}/bray_curtis_emperor.qzv')

<visualization: Visualization uuid: 7d930102-b3c4-4ef4-89f0-d1fd2c43e2b6>

In [21]:
Visualization.load(f'{workdir}/{output_dir}/jaccard_emperor.qzv')

<visualization: Visualization uuid: 3330f5bc-ff21-4b0b-b69c-e4fd8e012e9e>

## Kmerizer 

In [29]:
#!rm -r kmer-core-metrics

In [22]:
%%bash

qiime feature-table filter-seqs \
    --i-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-rep-seqs.qza \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_table.qza \
    --o-filtered-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_rep_seqs.qza

qiime kmerizer core-metrics \
    --i-sequences /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_rep_seqs.qza \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_table.qza \
    --p-sampling-depth 5000 \
    --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Valais_and_Lavaux23.tsv \
    --p-n-jobs 'auto' \
    --p-max-features 5000 \
    --output-dir valais_kmer_cm5000/

Saved FeatureData[Sequence] to: /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_rep_seqs.qza
Saved FeatureTable[Frequency] to: valais_kmer_cm5000/rarefied_table.qza
Saved FeatureTable[Frequency] to: valais_kmer_cm5000/kmer_table.qza
Saved SampleData[AlphaDiversity] to: valais_kmer_cm5000/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: valais_kmer_cm5000/shannon_vector.qza
Saved DistanceMatrix to: valais_kmer_cm5000/jaccard_distance_matrix.qza
Saved DistanceMatrix to: valais_kmer_cm5000/bray_curtis_distance_matrix.qza
Saved PCoAResults to: valais_kmer_cm5000/jaccard_pcoa_results.qza
Saved PCoAResults to: valais_kmer_cm5000/bray_curtis_pcoa_results.qza
Saved Visualization to: valais_kmer_cm5000/scatterplot.qzv


In [23]:
Visualization.load('valais_kmer_cm5000/scatterplot.qzv')

<visualization: Visualization uuid: 853f6ff4-f668-4dad-8182-5e965447ad60>

### Alpha Group Significance 

In [24]:
# metrics: 
alpha_metrics = ['shannon_vector', 'observed_features_vector', 'evenness_vector']

for m in alpha_metrics:
    ags, = alpha_group_significance(alpha_diversity = core_div_dict[m],
                               metadata = md)
    ags.save(output_dir + 'alpha_groupSig_{0}.qzv'.format(m))

/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  series[missing.index] = missing
/scratch/lfloerl/.condaenvs/qiime2-amplicon-

In [25]:
Visualization.load(f'{output_dir}/alpha_groupSig_evenness_vector.qzv')

<visualization: Visualization uuid: 102b2e14-e4db-4cff-a6d4-b9c040a5fb20>

In [26]:
Visualization.load(f'{output_dir}/alpha_groupSig_observed_features_vector.qzv')

<visualization: Visualization uuid: 10b408ff-53a0-443b-830b-789dc9710379>

In [27]:
Visualization.load(f'{output_dir}/alpha_groupSig_shannon_vector.qzv')

<visualization: Visualization uuid: bc5c4610-d5a9-471f-9702-ae9883b584a4>

### PERMANOVA

In [52]:
md_df = md.to_dataframe()
md_df['Location'].nunique()

31

In [47]:
formula = 'Location * Variety'

beta_metrics = ['bray_curtis', 'jaccard']

# PERMANOVA with core metrics 
for m in beta_metrics:
    adonis_res, = q2d.actions.adonis(
        distance_matrix=core_div_dict[m + '_distance_matrix'],
        metadata=md,
        formula=formula,
        n_jobs=3)
    adonis_res.save(output_dir + 'adonis_{0}.qzv'.format(m))
    

# PERMANOVA with kmer core metrics 
kmer_dir = 'valais_kmer_cm5000'
    
for m in beta_metrics:
    !qiime diversity adonis \
        --i-distance-matrix "{kmer_dir}/{m}_distance_matrix.qza" \
        --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Valais_and_Lavaux23.tsv \
        --p-formula "{formula}" \
        --p-n-jobs 5 \
        --o-visualization "{kmer_dir}/adonis_{m}.qzv"

/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  series[missing.index] = missing
/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan na

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: run_adonis.R /scratch/lfloerl/tmpdata/tmp8n_8rdld/dm.tsv /scratch/lfloerl/tmpdata/tmp8n_8rdld/md.tsv Location * Variety 999 3 /scratch/lfloerl/tmpdata/qiime2-temp-1rtdtfce/adonis.tsv

R version 4.3.3 (2024-02-29) 


This is vegan 2.6-8
Warnmeldung:
'adonis' ist veraltet.
Benutzen Sie stattdessen 'adonis2'
Siehe help("Deprecated") und help("vegan-deprecated"). 
/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  series[missing.index] = missing
/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan n

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: run_adonis.R /scratch/lfloerl/tmpdata/tmp_jffeqse/dm.tsv /scratch/lfloerl/tmpdata/tmp_jffeqse/md.tsv Location * Variety 999 3 /scratch/lfloerl/tmpdata/qiime2-temp-o6phgiet/adonis.tsv

R version 4.3.3 (2024-02-29) 


This is vegan 2.6-8
Warnmeldung:
'adonis' ist veraltet.
Benutzen Sie stattdessen 'adonis2'
Siehe help("Deprecated") und help("vegan-deprecated"). 


Saved Visualization to: valais_kmer_cm5000/adonis_bray_curtis.qzv
Saved Visualization to: valais_kmer_cm5000/adonis_jaccard.qzv


In [48]:
Visualization.load(f'{output_dir}/adonis_bray_curtis.qzv')

<visualization: Visualization uuid: 6d912c56-96f6-46a7-ad58-33f578ac1e57>

In [49]:
Visualization.load(f'{output_dir}/adonis_jaccard.qzv')

<visualization: Visualization uuid: 6027b137-dfce-4d57-bf9a-10a42a10d730>

In [50]:
Visualization.load(f'{kmer_dir}/adonis_bray_curtis.qzv')

<visualization: Visualization uuid: a8c2e198-3689-43e5-859a-1afb30ec453d>

In [51]:
Visualization.load(f'{kmer_dir}/adonis_jaccard.qzv')

<visualization: Visualization uuid: d7843b30-9b7a-4a98-b8e2-bbf5a4f1289d>

## Mantel 

In [53]:
%%bash 

div_dir='valais_cm5000'
kmer_dir='valais_kmer_cm5000'
output_dir="valais_mantel"

mkdir -p $output_dir

qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza \
    --i-dm2 $div_dir/bray_curtis_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/bray_curtis_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza \
    --i-dm2 $div_dir/jaccard_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/jaccard_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza \
    --i-dm2 $kmer_dir/bray_curtis_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/bray_curtis_kmer_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza \
    --i-dm2 $kmer_dir/jaccard_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/jaccard_kmer_mantel.qzv

Saved Visualization to: valais_mantel/bray_curtis_mantel.qzv
Saved Visualization to: valais_mantel/jaccard_mantel.qzv
Saved Visualization to: valais_mantel/bray_curtis_kmer_mantel.qzv
Saved Visualization to: valais_mantel/jaccard_kmer_mantel.qzv


In [54]:
Visualization.load('must_mantel/bray_curtis_mantel.qzv')

<visualization: Visualization uuid: a43ad935-0ce3-46b5-a080-63ed139699e6>

In [55]:
Visualization.load('must_mantel/jaccard_mantel.qzv')

<visualization: Visualization uuid: 84619478-fb62-4766-a0ad-92b67ca9589c>

In [56]:
Visualization.load('must_mantel/bray_curtis_kmer_mantel.qzv')

<visualization: Visualization uuid: aebe0b3e-828c-42ef-8853-4c81221077ef>

In [57]:
Visualization.load('must_mantel/jaccard_kmer_mantel.qzv')

<visualization: Visualization uuid: 454efe15-c736-40f5-8444-7a265115c946>

## Random Forest

In [63]:
%%bash 

# classify variety 
qiime sample-classifier classify-samples \
  --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Valais_and_Lavaux23.tsv \
  --m-metadata-column Variety \
  --p-random-state 666 \
  --p-n-jobs 6 \
  --output-dir ./Valais-classify-Variety/

Saved SampleEstimator[Classifier] to: ./Valais-classify-Variety/sample_estimator.qza
Saved FeatureData[Importance] to: ./Valais-classify-Variety/feature_importance.qza
Saved SampleData[ClassifierPredictions] to: ./Valais-classify-Variety/predictions.qza
Saved Visualization to: ./Valais-classify-Variety/model_summary.qzv
Saved Visualization to: ./Valais-classify-Variety/accuracy_results.qzv
Saved SampleData[Probabilities] to: ./Valais-classify-Variety/probabilities.qza
Saved Visualization to: ./Valais-classify-Variety/heatmap.qzv
Saved SampleData[TrueTargets] to: ./Valais-classify-Variety/training_targets.qza
Saved SampleData[TrueTargets] to: ./Valais-classify-Variety/test_targets.qza


In [64]:
Visualization.load('Valais-classify-Variety/accuracy_results.qzv')

<visualization: Visualization uuid: 35963127-b12c-4083-acea-0934726935d5>

In [65]:
Visualization.load('Valais-classify-Variety/heatmap.qzv')

<visualization: Visualization uuid: 6438484b-eee7-4c15-96f7-57911f9859b9>

In [66]:
%%bash 

# classify region 
qiime sample-classifier classify-samples \
  --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Valais_and_Lavaux23.tsv \
  --m-metadata-column Region \
  --p-random-state 666 \
  --p-n-jobs 6 \
  --output-dir ./Valais-classify-Region/

Saved SampleEstimator[Classifier] to: ./Valais-classify-Region/sample_estimator.qza
Saved FeatureData[Importance] to: ./Valais-classify-Region/feature_importance.qza
Saved SampleData[ClassifierPredictions] to: ./Valais-classify-Region/predictions.qza
Saved Visualization to: ./Valais-classify-Region/model_summary.qzv
Saved Visualization to: ./Valais-classify-Region/accuracy_results.qzv
Saved SampleData[Probabilities] to: ./Valais-classify-Region/probabilities.qza
Saved Visualization to: ./Valais-classify-Region/heatmap.qzv
Saved SampleData[TrueTargets] to: ./Valais-classify-Region/training_targets.qza
Saved SampleData[TrueTargets] to: ./Valais-classify-Region/test_targets.qza


In [67]:
Visualization.load('Valais-classify-Region/accuracy_results.qzv')

<visualization: Visualization uuid: 3379c858-7ef8-471e-9673-3b1d2f0f54eb>

In [68]:
Visualization.load('Valais-classify-Region/heatmap.qzv')

<visualization: Visualization uuid: cee5d152-f18c-4869-8456-58c8dde1948f>

<hr>

# Compare only Chasselas samples


In [22]:
#!rm -r Valais_Chasselas

In [23]:
%%bash 

# Define the variable for the variety
variety="Chasselas"

# Define the plot directory based on the variety
plot_dir="./Valais_Chasselas"
echo "Processing variety: $variety"

# Create directory for the variety and navigate into it
mkdir -p "$plot_dir"
pushd "$plot_dir" > /dev/null

# Filter table and rep seqs for the specified variety
qiime feature-table filter-samples \
  --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/Valais_Lavaux23_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Valais_and_Lavaux23.tsv \
  --p-where "[Variety]='$variety'" \
  --o-filtered-table must_harvest21_${variety}_filtered_table.qza

qiime feature-table filter-seqs \
  --i-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-rep-seqs.qza \
  --i-table must_harvest21_${variety}_filtered_table.qza \
  --o-filtered-data must_harvest21_${variety}_filtered_rep_seqs.qza

# Core diversity metrics
qiime diversity core-metrics \
  --i-table must_harvest21_${variety}_filtered_table.qza \
  --p-sampling-depth 5000 \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Valais_and_Lavaux23.tsv \
  --p-n-jobs 5 \
  --output-dir cm5000/

# Kmer diversity
qiime kmerizer core-metrics \
  --i-sequences must_harvest21_${variety}_filtered_rep_seqs.qza \
  --i-table must_harvest21_${variety}_filtered_table.qza \
  --p-sampling-depth 5000 \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Valais_and_Lavaux23.tsv \
  --p-n-jobs auto \
  --p-max-features 5000 \
  --output-dir kmer_cm5000/

# Perform Mantel tests and create mantel directory
mkdir -p mantel

qiime diversity mantel \
  --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_Valais_ITS.qza \
  --i-dm2 cm5000/bray_curtis_distance_matrix.qza \
  --p-intersect-ids \
  --o-visualization mantel/bray_curtis_mantel.qzv

qiime diversity mantel \
  --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_Valais_ITS.qza \
  --i-dm2 cm5000/jaccard_distance_matrix.qza \
  --p-intersect-ids \
  --o-visualization mantel/jaccard_mantel.qzv

qiime diversity mantel \
  --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_Valais_ITS.qza \
  --i-dm2 kmer_cm5000/bray_curtis_distance_matrix.qza \
  --p-intersect-ids \
  --o-visualization mantel/bray_curtis_kmer_mantel.qzv

qiime diversity mantel \
  --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_Valais_ITS.qza \
  --i-dm2 kmer_cm5000/jaccard_distance_matrix.qza \
  --p-intersect-ids \
  --o-visualization mantel/jaccard_kmer_mantel.qzv

# Return to the parent directory
popd > /dev/null


Processing variety: Chasselas
Saved FeatureTable[Frequency] to: must_harvest21_Chasselas_filtered_table.qza
Saved FeatureData[Sequence] to: must_harvest21_Chasselas_filtered_rep_seqs.qza
Saved FeatureTable[Frequency] to: cm5000/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: cm5000/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: cm5000/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: cm5000/evenness_vector.qza
Saved DistanceMatrix to: cm5000/jaccard_distance_matrix.qza
Saved DistanceMatrix to: cm5000/bray_curtis_distance_matrix.qza
Saved PCoAResults to: cm5000/jaccard_pcoa_results.qza
Saved PCoAResults to: cm5000/bray_curtis_pcoa_results.qza
Saved Visualization to: cm5000/jaccard_emperor.qzv
Saved Visualization to: cm5000/bray_curtis_emperor.qzv
Saved FeatureTable[Frequency] to: kmer_cm5000/rarefied_table.qza
Saved FeatureTable[Frequency] to: kmer_cm5000/kmer_table.qza
Saved SampleData[AlphaDiversity] to: kmer_cm5000/observed_features_vector.qza
Saved S

In [24]:
Visualization.load('Valais_Chasselas/mantel/bray_curtis_mantel.qzv')

<visualization: Visualization uuid: 9cc833eb-4964-42f7-b702-3b787eacd981>

In [25]:
Visualization.load('Valais_Chasselas/mantel/jaccard_mantel.qzv')

<visualization: Visualization uuid: 3e48b093-cfd6-485c-a200-7b95f3859a94>

In [26]:
Visualization.load('Valais_Chasselas/mantel/bray_curtis_kmer_mantel.qzv')

<visualization: Visualization uuid: 39a8c503-d82d-4f8a-a13b-f0d72fadd644>

In [27]:
Visualization.load('Valais_Chasselas/mantel/jaccard_kmer_mantel.qzv')

<visualization: Visualization uuid: d455bc0c-38ab-421d-8058-c1c1c4ec31b9>